Prétraitements :

- Supprimer les valeurs manquantes
- Utiliser la méthode du one-hot-encoding pour traiter les variables catégorielles
- Supprimer les colonnes liées au temps


Classification:

- Utilisez l’algorithme SVM créer un modèle afin de savoir si la personne a ou non fait un dépot long terme (colonne `y`)
- Évaluez les performances prédictives du modèle entrainé.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("https://gist.githubusercontent.com/jsz4n/b7ca11015784086788022a539935d0cf/raw/a8c3abf0a31f5c0df5e0ddd76fb9b289bac9bed1/bank-full.csv", sep=";")

In [3]:
df.head()

age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no     2143     yes   no   
1   44    technician   single  secondary      no       29     yes   no   
2   33  entrepreneur  married  secondary      no        2     yes  yes   
3   47   blue-collar  married    unknown      no     1506     yes   no   
4   33       unknown   single    unknown      no        1      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown    5   may       261         1     -1         0  unknown  no  
1  unknown    5   may       151         1     -1         0  unknown  no  
2  unknown    5   may        76         1     -1         0  unknown  no  
3  unknown    5   may        92         1     -1         0  unknown  no  
4  unknown    5   may       198         1     -1         0  unknown  no

In [4]:
df[["duration","y"]].boxplot(by="y")

<Axes: title={'center': 'duration'}, xlabel='[y]'>

on observe que plus la durée de l'appel est élevée plus on a de chance que `y` = `yes`

In [5]:
df.y.value_counts()

y
no     39922
yes     5289
Name: count, dtype: int64

In [6]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [7]:
categorical_features = [ 'job', 'marital', 'education', 'default', 'housing',
       'loan', 'contact', 'poutcome']
numerical_features = ['age','balance', 'pdays' , 'campaign', 'previous', 'duration']

In [8]:
X = df[(categorical_features+numerical_features)].copy()
y = df["y"].copy()
y = y.map(lambda x:x=="yes")

In [9]:
from imblearn.under_sampling import RandomUnderSampler

In [10]:
rus = RandomUnderSampler(random_state=42)
X_rs, y_rs = rus.fit_resample(X,y)

In [11]:
X_rs.shape, y_rs.shape

((10578, 14), (10578,))

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_rs, y_rs, test_size=.2)

# Preprocessing

In [14]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [15]:
scaler = StandardScaler()
scaler.fit(X_train[numerical_features])
scaler.set_output(transform = "pandas")
X_train_num = scaler.transform(X_train[numerical_features])
X_test_num = scaler.transform(X_test[numerical_features])

In [16]:
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
ohe.set_output(transform="pandas")
ohe.fit(X_train[categorical_features])
X_train_cat = ohe.transform(X_train[categorical_features])
X_test_cat = ohe.transform(X_test[categorical_features])

In [17]:
X_train_preprocessed = pd.concat([X_train_num, X_train_cat], axis=1)
X_test_preprocessed = pd.concat([X_test_num, X_test_cat], axis=1)

In [18]:
X_train_preprocessed.shape,\
X_test_preprocessed.shape

((8462, 38), (2116, 38))

# Modelisation

Creation du modèle

In [19]:
from sklearn.svm import SVC

In [20]:
svm = SVC()

In [21]:
svm.fit(X_train_preprocessed, y_train)

SVC()

Cross validation

Utiliser la validation croisé pour créer un graphique contenant les scores d’entrainement.


In [22]:
from sklearn.model_selection import cross_validate

metrics = cross_validate(svm, X_train_preprocessed, y_train, cv=10, scoring=["accuracy","f1", "precision", "recall"])

In [23]:
pd.DataFrame(metrics).plot(y=["test_recall", "test_precision", "test_f1", "test_accuracy"])

<Axes: >

Utilisation du grid search pour trouver les meilleur hyperparamètres

In [40]:
# from sklearn.model_selection import RandomizedSearchCV

In [24]:
from sklearn.model_selection import GridSearchCV

param_grid = {"kernel":["linear","poly", "rbf"], "C":[1,2,3], "probability":[True]}
grid_clf = GridSearchCV(svm, param_grid, cv=6, scoring="recall", n_jobs=4)

In [25]:
grid_clf.fit(X_train_preprocessed, y_train)

GridSearchCV(cv=6, estimator=SVC(), n_jobs=4,
             param_grid={'C': [1, 2, 3], 'kernel': ['linear', 'poly', 'rbf'],
                         'probability': [True]},
             scoring='recall')

In [33]:
grid_clf.cv_results_

{'mean_fit_time': array([28.92712502, 26.5147332 , 26.67158182, 34.16623116, 24.02859811,
        20.40307013, 30.99530466, 22.06640446, 19.39455938]),
 'std_fit_time': array([0.54025394, 0.5226947 , 0.23747676, 1.59260978, 1.67480585,
        1.07785474, 1.50372667, 0.72083259, 4.95869263]),
 'mean_score_time': array([0.33478479, 0.39222976, 0.71473555, 0.33030307, 0.27387226,
        0.40441008, 0.34623031, 0.27005919, 0.69520128]),
 'std_score_time': array([0.04105903, 0.04612064, 0.08703498, 0.03771993, 0.05923071,
        0.02458898, 0.06116714, 0.04453828, 0.43779289]),
 'param_C': masked_array(data=[1, 1, 1, 2, 2, 2, 3, 3, 3],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value=999999),
 'param_kernel': masked_array(data=['linear', 'poly', 'rbf', 'linear', 'poly', 'rbf',
                    'linear', 'poly', 'rbf'],
              mask=[False, False, False, False, False, False, False, False,
                  

In [32]:
grid_clf.best_params_

{'C': 2, 'kernel': 'rbf', 'probability': True}

In [34]:
svm = SVC(**grid_clf.best_params_)

In [36]:
svm.fit(X_train_preprocessed, y_train)

SVC(C=2, probability=True)

Évaluation du modèle

In [37]:
y_pred = svm.predict(X_test_preprocessed)

In [38]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [39]:
accuracy_score(y_test, y_pred),\
precision_score(y_test, y_pred),\
recall_score(y_test, y_pred),\
f1_score(y_test, y_pred)

(0.8241965973534972,
 np.float64(0.8070333633904418),
 np.float64(0.8499525166191833),
 np.float64(0.8279370952821462))

## Utilisation de grid search sur un decision tree

In [41]:
from sklearn.tree import DecisionTreeClassifier

In [45]:
dt = DecisionTreeClassifier()

param_grid = {"max_depth":[3,5,7,9], "min_samples_leaf":[1,5,10], "criterion":["gini","entropy"]}

In [46]:
grid_clf = GridSearchCV(estimator=dt, param_grid=param_grid, cv=10, scoring="recall", n_jobs=4)

In [47]:
grid_clf.fit(X_train_preprocessed, y_train)

/home/jeremie/Documents/Cours/MachineLearningSOPRA/.venv/lib/python3.11/site-packages/numpy/ma/core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=10, estimator=DecisionTreeClassifier(), n_jobs=4,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 7, 9],
                         'min_samples_leaf': [1, 5, 10]},
             scoring='recall')

In [48]:
grid_clf.cv_results_

{'mean_fit_time': array([0.03000934, 0.03076775, 0.0314786 , 0.04161754, 0.03989182,
        0.039677  , 0.05715711, 0.05450308, 0.05429921, 0.06634798,
        0.06708958, 0.06246917, 0.02895131, 0.02851214, 0.02862544,
        0.04305522, 0.04304521, 0.04274197, 0.06635206, 0.06073802,
        0.07493126, 0.07868414, 0.07328587, 0.07534299]),
 'std_fit_time': array([0.00293709, 0.00447049, 0.0047294 , 0.00232408, 0.00081588,
        0.00075642, 0.00441735, 0.00285555, 0.00351092, 0.00177002,
        0.00418496, 0.00095285, 0.00035269, 0.00079569, 0.00038302,
        0.00119765, 0.00078187, 0.0008799 , 0.0095484 , 0.00843717,
        0.00557148, 0.01434874, 0.0094958 , 0.01264589]),
 'mean_score_time': array([0.00795753, 0.00787838, 0.0073988 , 0.00655806, 0.0066839 ,
        0.00659919, 0.00688148, 0.0070003 , 0.00659883, 0.00671799,
        0.00668237, 0.00657895, 0.00643737, 0.00613627, 0.00664282,
        0.00643437, 0.00642188, 0.00637197, 0.00732553, 0.00738473,
        0.008859

In [49]:
grid_clf.best_params_

{'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 1}

In [50]:
dt = DecisionTreeClassifier(**grid_clf.best_params_)

In [51]:
dt.fit(X_train_preprocessed, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3)

In [53]:
y_pred = dt.predict(X_test_preprocessed)

In [55]:
dt.predict_proba(X_test_preprocessed[:5])

array([[0.86419753, 0.13580247],
       [0.09017341, 0.90982659],
       [0.3428023 , 0.6571977 ],
       [0.97935484, 0.02064516],
       [0.09017341, 0.90982659]])

In [54]:
accuracy_score(y_test, y_pred), recall_score(y_test, y_pred)

(0.7948960302457467, np.float64(0.8480531813865148))

Visualisation de la "decision boundary" (test)

In [29]:
# from sklearn.inspection import DecisionBoundaryDisplay

In [30]:
# cols = list(X_train_preprocessed.columns[:2])
# X_train_preprocessed[cols].shape

In [31]:
# dis = DecisionBoundaryDisplay.from_estimator(svm, X_train_preprocessed[cols],
#                                              response_method="predict",
#                                              xlabel = X_train_preprocessed.columns[0],
#                                              ylabel=X_train_preprocessed.columns[1])
# dis.plot()